# Spaceship Titanic

### 🚣‍♂️목적 
우주선 타이타닉 어떤 승객이 다른 차원으로 이동할지 예측

</br>

### 🧩특징 
우주 미스터리를 풀기 위해 여러분의 데이터 과학 기술이 필요한 2912년에 오신 것을 환영합니다.  
4광년 떨어진 곳에서 전송이 왔는데 상황이 좋지 않아요

우주선 타이타닉은 한 달 전에 발사된 성간 여객선입니다.  
거의 13,000명의 승객을 태운 이 배는 태양계에서 온 이민자들을  
근처의 별 주위를 도는 세 개의 새로이 거주할 수 있는 외계 행성으로 실어 나르는 처녀 항해를 시작했습니다.

첫 번째 목적지인 55호 암크리 E로 알파 센타우리를 도는 동안,  
조심성이 없는 우주선 타이타닉은 먼지 구름 속에 숨겨진 시공간 변칙과 충돌했습니다.  
안타깝게도, 그것은 1000년 전의 이름과 비슷한 운명을 만났습니다. 비록 그 배는 온전하게 유지되었지만,  
거의 절반의 승객들은 다른 차원으로 옮겨졌습니다!  

승무원을 구조하고 잃어버린 승객을 되찾는 것을 돕기 위해,  
여러분은 우주선의 손상된 컴퓨터 시스템에서 복구된 기록을 사용하여  
어떤 승객들이 이상 현상에 의해 이송되었는지 예측해야 합니다.

그들을 구하고 역사를 바꾸도록 도와주세요!

</br>

### 📔코드북
`train.csv` - 승객의 약 3분의 2(~8700)에 대한 개인 기록
- `PassengerId` - 각 승객의 고유 ID. 각 Id는 ggg_pp 형식을 취함  
    - gggg: 승객과 함께 여행하는 그룹  
    - pp: 그룹 내 승객의 번호  
    - 그룹의 사람들은 가족 구성원 또는 아닐 수 있음
- `HomePlanet` - 승객이 출발한 행성, 일반적으로 영구 거주지의 행성 
- `CryoSleep` - 승객이 항해 중 가사상태로 선출되었는지 여부
    - 극저온 수면 중인 승객들은 객실에 갇혀 있음 
- `Cabin` - 객실 번호(deck/num/side)
    - side: Port의 경우 P, Starboard의 경우 S
- `Destination` - 승객이 내릴 행선지
- `Age` - 승객의 연령
- `VIP` - 승객이 항해 중 특별 VIP 서비스 비용을 지불했는지 여부
- `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` - 승객들이 우주선 타이타닉의 많은 고급 편의시설 각각에 청구한 금액
- `Name` - 승객의 이름
- `Transported` - 승객이 다른 차원으로 운송되었는지 여부, target값

`test.csv` - 남은 승객의 1/3(약 4300명)에 대한 개인 기록
    - 승객에 대한 Transported 값을 예측 


[출처] <a href="https://www.kaggle.com/competitions/spaceship-titanic/overview/description">Spaceship Titanic</a>

# Step 1. Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.impute import SimpleImputer  # 결측치 대체
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier

import time
import warnings
warnings.filterwarnings('ignore')

# Step 2. Data Loading and Preperation

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

RANDOM_STATE = 12
FOLDS = 5
STRATEGE = 'median'

## Exploring Train Data
* 14개의 열과 8693개의 행이다.
* 2324개의 결측값을 포함한 119378개의 관측값이 있다.
* 12개의 열은 모두 결측값이 있고 CryoSleep이 결측값은 217개로 가장 많다.
* Transported는 train dataset에만 있는 target이다.

### Quick View of Train Data 
Below are the first 5 rows of train dataset:

In [3]:
display(train.head())
print(f"\33[92mtrain data의 행 개수: {train.shape[0]}")
print(f"train data의 열 개수: {train.shape[1]}")
print(f"train data의 관측값: {train.count().sum()}")
print(f"train data의 결측값: {train.isna().sum().sum()}")

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


train data의 행 개수: 8693
train data의 열 개수: 14
train data의 관측값: 119378
train data의 결측값: 2324


Column Wise missing values : 

In [4]:
print(f'\33[92m')
print(train.isna().sum().sort_values(ascending = False))


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64


Basic statistics of training data:

In [5]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,8514.0,28.827930,14.489021,0.0,19.0,27.0,38.0,79.0
RoomService,8512.0,224.687617,666.717663,0.0,0.0,0.0,47.0,14327.0
FoodCourt,8510.0,458.077203,1611.489240,0.0,0.0,0.0,76.0,29813.0
ShoppingMall,8485.0,173.729169,604.696458,0.0,0.0,0.0,27.0,23492.0
Spa,8510.0,311.138778,1136.705535,0.0,0.0,0.0,59.0,22408.0
VRDeck,8505.0,304.854791,1145.717189,0.0,0.0,0.0,46.0,24133.0


## Exploring Test Data
### Quick View of Test Data 
Below are the first 5 rows of test dataset:

In [6]:
display(test.head())
print(f"\33[92mtest data의 행 개수: {test.shape[0]}")
print(f"test data의 열 개수: {test.shape[1]}")
print(f"test data의 관측값: {test.count().sum()}")
print(f"test data의 결측값: {test.isna().sum().sum()}")

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


test data의 행 개수: 4277
test data의 열 개수: 13
test data의 관측값: 54484
test data의 결측값: 1117


Column Wise missing values : 

In [7]:
print(f'\33[92m')
print(test.isna().sum().sort_values(ascending = False))


FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


Basic statistics of training data:

In [8]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,4186.0,28.658146,14.179072,0.0,19.0,26.0,37.0,79.0
RoomService,4195.0,219.266269,607.011289,0.0,0.0,0.0,53.0,11567.0
FoodCourt,4171.0,439.484296,1527.663045,0.0,0.0,0.0,78.0,25273.0
ShoppingMall,4179.0,177.295525,560.821123,0.0,0.0,0.0,33.0,8292.0
Spa,4176.0,303.052443,1117.186015,0.0,0.0,0.0,50.0,19844.0
VRDeck,4197.0,310.710031,1246.994742,0.0,0.0,0.0,36.0,22272.0


### Quick View of Submission Data 

In [9]:
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


# Step 3. EDA

# Step 4. Data Pre-Processing

# Step 5. Modeling

# Step 6. Submission